In [3]:
import pandas as pd
import json
import os 
pd.set_option('display.max_rows', 20000)
pd.options.display.max_rows = 20000

In [4]:
os.getcwd()

'/Users/kellylee/code/challenges'

In [5]:
df = pd.read_csv('data/Vizion_Senior_Data_Analyst_Code_Challenge.csv')
df.shape

(196203, 16)

In [6]:
df.head()

,REFERENCE_ID,REFERENCE_CREATED_AT,REFERENCE_UPDATE_ID,DATA_UPDATED_AT,DESTINATION_PORT,ORIGIN_PORT,RAW_DESCRIPTION,EVENT_DESCRIPTION,EVENT_MODE,EVENT_LOCATION,EVENT_SOURCE,EVENT_TIMESTAMP,PLANNED,CONTAINER_ID,RELATED_VESSEL,CARRIER_NAME
0,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",departure,Vessel departure from transshipment port,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Colombo"",\n ...",ais,2024-06-29 07:15:06.000 Z,False,WHLU0639965,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai
1,09dbaf79-6a76-4efd-a511-663cf9cdd1c2,2024-06-21 22:16:30.000 Z,511e6a19-f4a8-40f0-a7c4-5e1e4d5e90ca,2024-07-30 10:08:45.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Itapoa"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Shenzhen"",\n ...",berth,Vessel arrived,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Hong Kong"",\n...",ais,2024-06-18 07:44:20.000 Z,False,PIDU4240363,"{\n ""imo"": ""9793959"",\n ""mmsi"": ""477233200"",...",Pacific International Lines
2,07bedccb-b83f-4d87-bc55-8ad54b9755c1,2024-06-19 07:00:14.000 Z,03d9e401-257d-4c47-af37-a4ee52cbe58e,2024-07-15 07:01:01.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Norfolk"",\n ...","{\n ""bic_cd"": null,\n ""city"": ""Chennai"",\n ...",Inbound/Outbound Laden Ctnr Move From One Pier...,Gate out,Truck,"{\n ""bic_cd"": null,\n ""city"": null,\n ""coun...",carrier,2024-06-29 00:50:00.000 Z,False,WHLU0639965,"{\n ""imo"": null,\n ""mmsi"": null,\n ""vessel""...",Wan Hai
3,924b8c26-7f7a-419b-9d81-c4857a918b8e,2024-06-17 16:50:59.000 Z,2c5eb28d-67fb-4114-9ee3-47cafc7620f7,2024-08-12 16:34:34.000 Z,"{\n ""bic_cd"": null,\n ""city"": ""Mobile"",\n ""...","{\n ""bic_cd"": null,\n ""city"": ""Rotterdam"",\n...",departure,Vessel departed,Vessel,"{\n ""bic_cd"": null,\n ""city"": ""Veracruz"",\n ...",ais,2024-06-06 11:14:46.000 Z,False,SUDU5331459,"{\n ""imo"": ""9535163"",\n ""mmsi"": ""477346500"",...",Hamburg-Süd
4,001dd4c8-ac21-4042-b762-b6650815b649,2024-06-13 07:00:23.000 Z,bb20cbf7-52b1-48c7-bf10-776dced41dea,2024-07-03 20:19:47.000 Z,"{\n ""city"": ""Savannah"",\n ""country"": ""United...","{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",Laden Ctnr Loaded On Board Vessel.(Mother Vess...,Loaded on vessel at origin port,Vessel,"{\n ""city"": ""Shenzhen"",\n ""country"": ""China""...",carrier,2024-06-15 01:12:00.000 Z,False,WHLU5809684,"{\n ""imo"": ""9555101"",\n ""mmsi"": ""563172700"",...",Wan Hai


In [7]:
df.columns

Index(['REFERENCE_ID', 'REFERENCE_CREATED_AT', 'REFERENCE_UPDATE_ID',
       'DATA_UPDATED_AT', 'DESTINATION_PORT', 'ORIGIN_PORT', 'RAW_DESCRIPTION',
       'EVENT_DESCRIPTION', 'EVENT_MODE', 'EVENT_LOCATION', 'EVENT_SOURCE',
       'EVENT_TIMESTAMP', 'PLANNED', 'CONTAINER_ID', 'RELATED_VESSEL',
       'CARRIER_NAME'],
      dtype='object')

# Task 1
There are eight core events expected to occur for each journey identified by “reference_id”. What are the rates for how often these events are present for each shipment? You may need to consider whether the container journey is completed for more recent shipments.

- overall
- overall per ocean carrier
- per each of the 8 core events overall
- per each of the 8 core events per ocean carrier

In [8]:
core_events = ['Gate out from origin port', 
               'Gate in at origin port',
               'Loaded on vessel at origin port',
               'Vessel departure from origin port',
               'Vessel arrived at destination port',
               'Discharged from vessel at destination port',
               'Gate out from destination port',
               'Gate in empty return']

In [9]:
idx_of_core_events = df['EVENT_DESCRIPTION'].isin(core_events)
len(idx_of_core_events)

196203

Here, I assume that shipment is equivalent to journey. So the question is about the rates for how often the eight core events are present for each unique journey.

In [10]:
total_unique_journey = len(df['REFERENCE_ID'].unique())

core_events_and_all_reference_id = df.loc[idx_of_core_events, ['REFERENCE_ID', 'EVENT_DESCRIPTION']]
total_core_events_for_unique_journey = len(core_events_and_all_reference_id.drop_duplicates())

rate_overall = total_core_events_for_unique_journey / (total_unique_journey*8)

print(f"The rates for all 8 core events present for each shipment/journey: {round(rate_overall*100, 2)}  %")

The rates for all 8 core events present for each shipment/journey: 79.26  %


In [11]:
total_unique_journey_per_carrier = df.groupby('CARRIER_NAME').nunique('REFERENCE_ID')['REFERENCE_ID']

all_core_events_by_carriers = df.loc[idx_of_core_events, ['REFERENCE_ID', 'EVENT_DESCRIPTION', 'CARRIER_NAME']]
unique_core_events_by_carrier = all_core_events_by_carriers.drop_duplicates()
total_unique_core_events_by_carrier = unique_core_events_by_carrier.groupby('CARRIER_NAME').count()['REFERENCE_ID']

rate_overall_per_carrier = total_unique_core_events_by_carrier / (total_unique_journey_per_carrier*8)

print(f"The rates for all 8 core events present for each shipment/journey per ocean carrier: \n\
{round(rate_overall_per_carrier, 2).to_string(header=False)}")


The rates for all 8 core events present for each shipment/journey per ocean carrier: 
ANL                              0.93
APL                              0.80
CMA-CGM                          0.88
COSCO                            0.64
Hamburg-Süd                      0.78
Hapag Lloyd                      0.84
Hueng A                          0.74
Hyundai Merchant Marine (HMM)    0.81
Korean Marine Transport          0.90
MSC                              0.72
Maersk                           0.78
Matson                           0.61
Namsung Shipping                 0.92
OOCL                             0.80
Ocean Network Express (ONE)      0.94
Pacific International Lines      0.93
RCL Group                        0.85
SM Lines                         0.96
Safmarine                        0.82
Sealand                          0.80
Sinokor                          0.64
Wan Hai                          0.62
Westwood                         0.25
Yang Ming                        0.78
ZI

In [12]:
count_of_core_event_occurence = df[idx_of_core_events].groupby('EVENT_DESCRIPTION').nunique()['REFERENCE_ID']
rate_per_core_event = count_of_core_event_occurence / total_unique_journey

print(f"The rates for each core event present for each shipment/journey: \n\
{round(rate_per_core_event, 2).to_string(header=False)}")

The rates for each core event present for each shipment/journey: 
Discharged from vessel at destination port    0.77
Gate in at origin port                        0.83
Gate in empty return                          0.56
Gate out from destination port                0.78
Gate out from origin port                     0.64
Loaded on vessel at origin port               0.87
Vessel arrived at destination port            0.97
Vessel departure from origin port             0.92


In [13]:
ct_of_core_event_per_carrier = df[idx_of_core_events].groupby(['CARRIER_NAME',
                                                               'EVENT_DESCRIPTION']).nunique()['REFERENCE_ID']
rate_per_core_event_per_carrier = ct_of_core_event_per_carrier / total_unique_journey_per_carrier

print(f"The rates for each core event present for each shipment/journey per ocean carrier: \n\n\
{round(rate_per_core_event_per_carrier, 2).to_string(header=False)}")

The rates for each core event present for each shipment/journey per ocean carrier: 

ANL                            Discharged from vessel at destination port    0.92
                               Gate in at origin port                        1.00
                               Gate in empty return                          0.71
                               Gate out from destination port                0.92
                               Gate out from origin port                     0.92
                               Loaded on vessel at origin port               1.00
                               Vessel arrived at destination port            0.96
                               Vessel departure from origin port             1.00
APL                            Discharged from vessel at destination port    0.72
                               Gate in at origin port                        0.96
                               Gate in empty return                          0.32
             

# Task 2
Import dwell time is calculated as the time from [Discharged from vessel at destination port] to [Gate out from destination port], giving you the time a container dwelled in a terminal before an out gate to the consignee. For shipments that have both of these events can you determine the import dwell time per shipment?

In [14]:
df['EVENT_TIMESTAMP'] = pd.to_datetime(df['EVENT_TIMESTAMP'])

In [15]:
two_events = ['Discharged from vessel at destination port', 'Gate out from destination port']

df_two_events = df.loc[df['EVENT_DESCRIPTION'].isin(two_events),
                       ['EVENT_DESCRIPTION', 'EVENT_TIMESTAMP', 'REFERENCE_ID']].drop_duplicates()
df_two_events = df_two_events.dropna(subset='EVENT_TIMESTAMP').sort_values('EVENT_TIMESTAMP')

ct_of_events_per_journey = df_two_events.groupby('REFERENCE_ID').agg({'EVENT_DESCRIPTION': lambda x: x.nunique()})

qualified_journey_w_two_events = ct_of_events_per_journey[ct_of_events_per_journey['EVENT_DESCRIPTION']==2].index

In [16]:
qualified_journey = df_two_events[df_two_events['REFERENCE_ID'].isin(qualified_journey_w_two_events)]
qualified_journey_min_max = qualified_journey.groupby(['REFERENCE_ID','EVENT_DESCRIPTION']).agg({'min', 'max'}).reset_index()
qualified_journey_min_max.columns = ['ID', 'EVENT', 'MAX_TIME', 'MIN_TIME']
qualified_journey_min_max.head()

,ID,EVENT,MAX_TIME,MIN_TIME
0,000368a8-f6ee-4b64-8703-962cb95d50f7,Discharged from vessel at destination port,2024-07-02 13:30:00+00:00,2024-07-10 20:15:00+00:00
1,000368a8-f6ee-4b64-8703-962cb95d50f7,Gate out from destination port,2024-07-02 18:00:00+00:00,2024-07-26 22:07:00+00:00
2,0003c20e-d825-4be4-b10c-c97ba64a5d14,Discharged from vessel at destination port,2024-03-26 13:41:00+00:00,2024-03-26 13:41:00+00:00
3,0003c20e-d825-4be4-b10c-c97ba64a5d14,Gate out from destination port,2024-04-01 14:08:00+00:00,2024-04-01 14:08:00+00:00
4,0004450d-97e2-47cd-be1f-bb962adea35a,Discharged from vessel at destination port,2024-07-20 03:08:00+00:00,2024-07-20 03:08:00+00:00


In [17]:
# calculation is max of [Gate out from destination port] subtract the min of [Discharged from vessel at destination port]
max_of_gate_out = qualified_journey_min_max.loc[qualified_journey_min_max['EVENT']==two_events[1], 
                                                ['ID', 'MAX_TIME']]
min_of_discharge = qualified_journey_min_max.loc[qualified_journey_min_max['EVENT']==two_events[0], 
                                                ['ID', 'MIN_TIME']]
min_max_combined = max_of_gate_out.merge(min_of_discharge, on = 'ID')
min_max_combined['import_dwell_time'] = min_max_combined['MAX_TIME'] - min_max_combined['MIN_TIME']
import_dwell_time_per_journey = min_max_combined.drop(columns = ['MAX_TIME', 'MIN_TIME'])
import_dwell_time_per_journey

,ID,import_dwell_time
0,000368a8-f6ee-4b64-8703-962cb95d50f7,-9 days +21:45:00
1,0003c20e-d825-4be4-b10c-c97ba64a5d14,6 days 00:27:00
2,0004450d-97e2-47cd-be1f-bb962adea35a,4 days 01:41:00
3,00046ba3-62c5-40a3-b3ef-5622ab09e113,-5 days +02:53:00
4,0004dab1-0da0-42ad-b644-dc29ee66b537,-1 days +15:07:00
5,0004f67f-ee3b-411e-b5f9-7709f2b60748,1 days 18:46:00
6,0005c84d-bc4f-43b8-a24e-e6af7d226be1,9 days 01:53:00
7,00081cf2-4c1c-4de1-92d6-9b92adf59ab5,5 days 00:00:00
8,000893fa-6229-40ee-89a7-7984e21d7bc3,2 days 04:05:00
9,00093852-c8d1-4865-80fa-710bd517b7dd,2 days 10:21:00


# Task 3
Are origin and destination port values changing throughout the journey of a container? If so, how many container journeys are experiencing this?

In [18]:
df_valid_origin = df.loc[pd.notna(df['ORIGIN_PORT']), ['REFERENCE_ID', 'ORIGIN_PORT']].drop_duplicates()

num_of_origin_change = len(df_valid_origin.groupby('REFERENCE_ID').nunique() > 1)

print(f"{num_of_origin_change} containers/reference_id experienced changes of the \
origin ports throughout their journeys.")


577 containers/reference_id experienced changes of the origin ports throughout their journeys.


In [19]:
df_valid_destination = df.loc[pd.notna(df['DESTINATION_PORT']), ['REFERENCE_ID', 'DESTINATION_PORT']].drop_duplicates()

num_of_destination_change = len(df_valid_destination.groupby('REFERENCE_ID').nunique() > 1)

print(f"{num_of_destination_change} containers/reference_id experienced changes of the \
destination ports throughout their journeys.")


576 containers/reference_id experienced changes of the destination ports throughout their journeys.


# Task 4
Create a matrix that shows all events reported (not just the 8 core ocean). For these events show:

- which carriers report the event
- if the event has been reported as an estimate, an actual, or both

In [20]:
base = df.groupby(['EVENT_DESCRIPTION', 'CARRIER_NAME', 'PLANNED']).count()['REFERENCE_ID'].reset_index()
# print(type(base))
matrix_form = base.pivot(index = ['EVENT_DESCRIPTION', 'CARRIER_NAME'], columns = 'PLANNED', values = 'REFERENCE_ID')
matrix_form.columns = ['ESTIMATED (planned)', 'ACTUAL (not planned)']
matrix_form

ESTIMATED (planned)  \
EVENT_DESCRIPTION                                  CARRIER_NAME                                         
Arrival by Truck at Port terminal                  Yang Ming                                      1.0   
Arrival by Vessel at Port terminal                 Yang Ming                                      3.0   
Available for release / delivery                   ANL                                            8.0   
                                                   COSCO                                          NaN   
                                                   OOCL                                          43.0   
                                                   Sealand                                        3.0   
                                                   ZIM                                           76.0   
BARGE FULL DISCHARGING                             Namsung Shipping                               1.0   
BARGE FULL LOADING                                 Namsung Shipping                               7.0   
Barge arrived                                      ANL                                            NaN   
                                                   APL                                            NaN   
                                                   CMA-CGM                                        NaN   
                                                   Hyundai Merchant Marine (HMM)                 32.0   
                                                   Maersk                                         NaN   
                                                   OOCL                                          37.0   
                                                   Sealand                                        NaN   
                                                   Yang Ming                                     23.0   
Barge arrived at destination                       ANL                                            NaN   
Barge departed                                     ANL                                            NaN   
                                                   APL                                            NaN   
                                                   CMA-CGM                                        NaN   
                                                   Hyundai Merchant Marine (HMM)                 32.0   
                                                   Maersk                                         NaN   
                                                   OOCL                                          37.0   
                                                   Sealand                                        NaN   
                                                   Yang Ming                                     23.0   
CONTAINER ON HOLD                                  Matson                                        31.0   
CONTAINER ON HOLD - Payment Status                 Matson                                        12.0   
Car spot                                           Safmarine                                      NaN   
Cargo cutoff date                                  COSCO                                         28.0   
Carrier release                                    ANL                                            3.0   
                                                   APL                                           41.0   
                                                   MSC                                           47.0   
                                                   OOCL                                         231.0   
                                                   ZIM                                          134.0   
Construction Placement                             OOCL                                           6.0   
Container available for pickup                     ANL                                      

# Bonus 1
A data enrichment process enhances the vessel data. What gaps still exist regarding the 3 vessel data points of [imo], [mmsi], [vessel]?

In [21]:
def parse_vessel_data(data, to_check):
    x = str(data)
    if '{' in x:
        xx = json.loads(x)
        if to_check in xx.keys():
            if xx[to_check] != None:
                return False
            else:
                return True
        else:
            return True
    else:
        return True
df['imo_missing'] = df['RELATED_VESSEL'].apply(lambda x: parse_vessel_data(x, 'imo'))
df['mmsi_missing'] = df['RELATED_VESSEL'].apply(lambda x: parse_vessel_data(x, 'mmsi'))
df['vessel_missing'] = df['RELATED_VESSEL'].apply(lambda x: parse_vessel_data(x, 'vessel'))

print(f"Gap regarding [imo]: {sum(df['imo_missing'])} missing data ({int(sum(df['imo_missing'])/df.shape[0]*100)}% of the whole dataset)")
print(f"Gap regarding [mmsi]: {sum(df['mmsi_missing'])} missing data ({int(sum(df['imo_missing'])/df.shape[0]*100)}% of the whole dataset)")
print(f"Gap regarding [vessel]: {sum(df['vessel_missing'])} missing data ({int(sum(df['imo_missing'])/df.shape[0]*100)}% of the whole dataset)")

Gap regarding [imo]: 63559 missing data (32% of the whole dataset)
Gap regarding [mmsi]: 64014 missing data (32% of the whole dataset)
Gap regarding [vessel]: 62444 missing data (32% of the whole dataset)


# Bonus 2
Calculate import dwell time per destination port to determine which ports may perform better in regard to container throughput.

In [62]:
two_events = ['Discharged from vessel at destination port', 'Gate out from destination port']

dest_port_df = df.loc[df['EVENT_DESCRIPTION'].isin(two_events),
                       ['EVENT_DESCRIPTION', 'EVENT_TIMESTAMP', 'REFERENCE_ID', 'DESTINATION_PORT']].drop_duplicates()

dest_port_df = dest_port_df.dropna(subset='EVENT_TIMESTAMP').sort_values('EVENT_TIMESTAMP')

dest_port_df['PORT_NAME'] = dest_port_df['DESTINATION_PORT'].apply(lambda x: json.loads(x)['unlocode']
                                                                  if '{' in str(x) else x)

base_gate_out = dest_port_df.loc[dest_port_df['EVENT_DESCRIPTION']==two_events[1]]
max_gate_out = base_gate_out.groupby(['PORT_NAME', 'REFERENCE_ID']).max()['EVENT_TIMESTAMP']
max_gate_out = max_gate_out.reset_index().rename(columns={'EVENT_TIMESTAMP':'LAST_GATE_OUT'})

base_discharged = dest_port_df.loc[dest_port_df['EVENT_DESCRIPTION']==two_events[0]]
min_discharged = base_discharged.groupby(['PORT_NAME', 'REFERENCE_ID']).min()['EVENT_TIMESTAMP']
min_discharged = min_discharged.reset_index().rename(columns={'EVENT_TIMESTAMP':'FIRST_DISCHARGED'})
min_discharged.head()

bonus_all_combined = max_gate_out.merge(min_discharged, on = ['PORT_NAME', 'REFERENCE_ID'])

bonus_all_combined['import_dwell_time'] = bonus_all_combined['LAST_GATE_OUT'] - bonus_all_combined['FIRST_DISCHARGED']
import_dwell_time_per_dest_port = bonus_all_combined.groupby('PORT_NAME').mean()[['import_dwell_time']]
import_dwell_time_per_dest_port = import_dwell_time_per_dest_port.rename(columns = {'import_dwell_time': 'Average Import Dwell Time'})
import_dwell_time_per_dest_port.sort_values('Average Import Dwell Time', ascending = True)

/var/folders/t1/3gbhcbmx5554jx374y2d_2d40000gn/T/ipykernel_8816/3585497476.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  import_dwell_time_per_dest_port = bonus_all_combined.groupby('PORT_NAME').mean()[['import_dwell_time']]


,Average Import Dwell Time
PORT_NAME,
AUFRE,0 days 05:07:00
MYLPK,0 days 14:24:00
UYMVD,0 days 19:16:00
LKCMB,0 days 21:23:00
FRLEH,0 days 23:22:30
USPHL,1 days 00:00:00
DEBRV,1 days 03:53:00
CNQIN,1 days 07:40:00
CNSHK,1 days 14:47:00
